In [8]:
pip install rich

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
#!pip install -qU langchain-teddynote

from langchain_teddynote import logging

logging.langsmith("news.chatbot.ipynb")

LangSmith 추적을 시작합니다.
[프로젝트명]
news.chatbot.ipynb


In [5]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

c:\Users\heejunekim\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [14]:
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/mnews/article/001/0015623016?rc=N&ntype=RANKING",),
    bs_kwargs= dict(
        parse_only = bs4.SoupStrainer(
            "div",
            attrs = {"class" : ["newsct_article _article_body", "meida_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(metadata={'source': 'https://n.news.naver.com/mnews/article/001/0015623016?rc=N&ntype=RANKING'}, page_content='\n\n배터리공장 건설 현장서 체포된 지 8일만…"美 재입국 시 불이익 없게"중국 10명·일본 3명·인도네시아 1명 등 외국인 14명도 도착\n\n\n\n한국으로 향하는 석방 근로자 탑승 전세기(애틀랜타=연합뉴스) 장하나 기자 = 미국 이민 당국에 구금됐다가 풀려난 한국인 근로자들이 탑승한 대한항공 전세기가 11일(현지시간) 미국 조지아주 하츠필드-잭슨 애틀랜타 국제공항에서 이륙하고 있다. 2025.9.12 hanajjang@yna.co.kr    (서울=연합뉴스) 임성호 기자 = 미국 이민당국에 의해 조지아주에 구금됐다 풀려난 한국인 근로자 등 330명이 12일 오후 인천국제공항에 도착했다.    항공업계에 따르면 지난 11일(현지시간) 오전 11시 38분께 하츠필드-잭슨 애틀랜타 국제공항에서 출발한 대한항공 전세기 KE9036편은 이날 오후 3시 23분께 인천공항 활주로에 착륙했다.    이 항공편에는 한국인 총 316명(잔류 선택 1명 제외)과 외국 국적자 14명(중국 10명, 일본 3명, 인도네시아 1명) 등 조지아주 남부 포크스턴 구금시설 등에 억류됐던 근로자 총 330명이 탑승했다.    이번 사태를 수습하기 위해 미국을 찾은 박윤주 외교부 1차관과 김동명 LG에너지솔루션 대표를 비롯한 정부·기업 관계자 및 의료진 등 21명도 동승해 총 탑승 인원은 351명이다.    풀려난 한국인들은 체포·구금된 지 8일 만에 고국 땅을 밟게 됐다.    이들은 입국 절차를 거친 뒤 인천공항 제2여객터미널 출국장에서 가족 및 친지와 상봉할 예정이다.    앞서 미국 이민당국은 지난 4일 조지아주 엘러벨의 현대차그룹-LG에너지솔루션 배터리 합작공장 건설 현장에서 대대적인 기습 단속을 벌여 LG에너지솔루션과 협력사 직원 등 한국인 317명을 포함해 총 4

In [ ]:
text_splitter = RecursiveCharacterTextSplitter (chunk_size = 1000, chunk_overlap = 50)

splits = text_splitter.split_documents(docs)
len(splits)

4

In [19]:
print(splits[1])

page_content='한국으로 향하는 석방 근로자 탑승 전세기(애틀랜타=연합뉴스) 장하나 기자 = 미국 이민 당국에 구금됐다가 풀려난 한국인 근로자들이 탑승한 대한항공 전세기가 11일(현지시간) 미국 조지아주 하츠필드-잭슨 애틀랜타 국제공항에서 이륙하고 있다. 2025.9.12 hanajjang@yna.co.kr    (서울=연합뉴스) 임성호 기자 = 미국 이민당국에 의해 조지아주에 구금됐다 풀려난 한국인 근로자 등 330명이 12일 오후 인천국제공항에 도착했다.    항공업계에 따르면 지난 11일(현지시간) 오전 11시 38분께 하츠필드-잭슨 애틀랜타 국제공항에서 출발한 대한항공 전세기 KE9036편은 이날 오후 3시 23분께 인천공항 활주로에 착륙했다.    이 항공편에는 한국인 총 316명(잔류 선택 1명 제외)과 외국 국적자 14명(중국 10명, 일본 3명, 인도네시아 1명) 등 조지아주 남부 포크스턴 구금시설 등에 억류됐던 근로자 총 330명이 탑승했다.    이번 사태를 수습하기 위해 미국을 찾은 박윤주 외교부 1차관과 김동명 LG에너지솔루션 대표를 비롯한 정부·기업 관계자 및 의료진 등 21명도 동승해 총 탑승 인원은 351명이다.    풀려난 한국인들은 체포·구금된 지 8일 만에 고국 땅을 밟게 됐다.    이들은 입국 절차를 거친 뒤 인천공항 제2여객터미널 출국장에서 가족 및 친지와 상봉할 예정이다.    앞서 미국 이민당국은 지난 4일 조지아주 엘러벨의 현대차그룹-LG에너지솔루션 배터리 합작공장 건설 현장에서 대대적인 기습 단속을 벌여 LG에너지솔루션과 협력사 직원 등 한국인 317명을 포함해 총 475명을 체포했다. 당국은 이들이 미국에 불법적으로 입국했거나, 체류 자격 요건을 위반한 채 불법적으로 일하고 있었다고 설명했다.    우리 정부는 긴급 대책회의를 열어 대응에 나섰고, 현지에서는 주애틀랜타 총영사관 소속 영사가 사태 발생 사흘 차인 6일 오전부터 구금시설을 찾아 수감된 한국인들을 면담했다.    이후 석방 교섭이 속도를 내면서

In [ ]:
vectorstore = FAISS.from_documents(documents = splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()